In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from google.colab import files
import io
from PIL import Image

# Import different base models
from tensorflow.keras.applications import VGG16, ResNet50, MobileNet

def load_model(model_name):
    if model_name == 'VGG16':
        from tensorflow.keras.applications.vgg16 import preprocess_input
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'ResNet50':
        from tensorflow.keras.applications.resnet50 import preprocess_input
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'MobileNet':
        from tensorflow.keras.applications.mobilenet import preprocess_input
        base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    else:
        return None, None

    # Freeze all layers in the base model
    for layer in base_model.layers:
        layer.trainable = False

    # Add new layers
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(128, activation='relu')(x)
    output = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

    return model, preprocess_input

# Function to load and preprocess the image
def load_and_preprocess_image(img, preprocess_input):
    img = img.resize((224, 224))  # Resize image to 224x224
    img_array = img_to_array(img)
    img_array_expanded = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array_expanded)  # Preprocess the image as required by the model

# Function to predict the class of the image
def predict_image_class(model, img, preprocess_input):
    processed_img = load_and_preprocess_image(img, preprocess_input)
    prediction = model.predict(processed_img)
    return "Motorcycle" if prediction[0] > 0.5 else "Car"

# Prompt the user to upload an image file and select a model
model_name = 'ResNet50'  # or 'ResNet50', 'MobileNet', VGG16
model, preprocess_input = load_model(model_name)
uploaded = files.upload()

if uploaded and model is not None:
    for fn in uploaded.keys():
        path = io.BytesIO(uploaded[fn])
        img = Image.open(path)
        result = predict_image_class(model, img, preprocess_input)
        print(f"This image is a: {result}")


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Saving download (1).jpg to download (1) (1).jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
This image is a: Motorcycle
